### 1) Principais bibliotecas

In [1]:
import requests
import pandas as pd
import numpy as np
import time 
import json                     # para salvar os dados
from bs4 import BeautifulSoup
# Manipulação com o Selenium,
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.firefox.options import Options
from selenium.common.exceptions import WebDriverException, ElementClickInterceptedException, NoSuchElementException, StaleElementReferenceException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
# Necessário para iniciar o Chrome.
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import time 
import re, time, os 

### 2) Lê o arquivo csv contendo os links e insere dentro de uma lista.

In [4]:
import csv 
filename = "links_sales_florianopolis.csv"
links_rent_florianopolis = []
# open the csv file in read mode
with open(filename, 'r') as file:
    # create a csv reader object
    reader = csv.DictReader(file)
    # iterate over each row of the csv file
len(links_rent_florianopolis)

0

### 3) Define a posição inicial e final para ser percorrida no loop. <font color = yellow> (alterar aqui) </font>

In [4]:
initial_pos = 2644 
last_pos = 6781

In [5]:
results = []
wait_time = 0.5
list_position = initial_pos
#! We can set an range of values to get inside the list.
for link in links_rent_florianopolis[initial_pos:last_pos]: 
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)
    driver.get(str(link))
    driver.maximize_window() 
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    #WebDriverWait(driver, wait_time).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="cookie-notifier-cta"]'))).click()
    time.sleep(wait_time)

    if "Você está vendo esta página porque o imóvel que buscava foi alugado ou está indisponível." in soup.get_text():
        list_position += 1
        print(list_position)
        driver.quit() 
    else: 
        time.sleep(wait_time)
        try:
            title = soup.find('h1', 'title__title js-title-view').get_text().strip()
        except AttributeError:
            title = "N/I"
        try:
            complete_address = soup.find('p', 'title__address js-address').get_text().strip()
        except AttributeError:
            complete_address = "N/I"
        try:
            area = soup.find('li', 'features__item features__item--area js-area').get_text().strip()
        except AttributeError:
            area = "N/I"
        try:
            rooms = soup.find('li', 'features__item features__item--bedroom js-bedrooms').get_text().strip()
        except AttributeError:
            rooms = "N/I"
        try:
            baths = soup.find('li', 'features__item features__item--bathroom js-bathrooms').get_text().strip()
        except AttributeError:
            baths = "N/I"
        try:
            garage = soup.find('li', 'features__item features__item--parking js-parking').get_text().strip()
        except AttributeError:
            garage = "N/I"  
        # Extracting Prices
        try:
            price_sale = soup.find('h3', 'price__price-info js-price-sale').get_text().strip()
        except AttributeError:
            price_sale = "N/I"
        try:
            condominium_fee = soup.find('span', 'price__list-value condominium js-condominium').get_text().strip()
        except AttributeError:
            condominium_fee = "N/I"
        try:
            iptu = soup.find('span', 'price__list-value iptu js-iptu').get_text().strip()
        except AttributeError:
            iptu = "N/I"
        try: 
            ad_description = soup.find('p', 'description__text').get_text().strip()
        except AttributeError:
            ad_description = "N/I"

        # Splitting the address 
        add_list = re.split(',|-', complete_address)
        add_list = [item.strip() for item in add_list]
        if len(add_list) == 2: 
            city, state = add_list
            neibhood = "N/I"
            address = 'N/I'
            number = 'N/I'
        if len(add_list) == 3: 
            neibhood, city, state = add_list
            address = 'N/I'
            number = 'N/I'
        if len(add_list) == 4: 
            address, neibhood, city, state = add_list
            number = "N/I"
        elif len(add_list) == 5:
            address, number, neibhood, city, state = add_list

        """
        Tentei fazer por XPath, porém tem anuncios que tem o mesmo xpath mas levam a outro site. 
        Para fazer By.ClassName, usar como value o final da casa (se usar tudo nao funciona)
        """
        try:
            button_charac = driver.find_element(by=By.CLASS_NAME, value = 'js-amenities-button')
            ActionChains(driver).move_to_element(button_charac).perform()
            attempts = 0
            while attempts < 1:
                try:
                    button_charac.click()
                    time.sleep(wait_time) 
                    characteristics = soup.find('ul', 'amenities__list').get_text().strip()
                    break
                except StaleElementReferenceException as exception:
                    characteristics = "N/I"
        except ElementClickInterceptedException as exception: 
            characteristics = "N/I"
        except NoSuchElementException: 
            characteristics = "N/I"       

            
        row = {'Título': title, 'Descrição': ad_description, 'Endereço': address, 'Número': number, 'Bairro': neibhood, 'Cidade': city, 'Estado': state, 'Área': area, 'Quartos': rooms, 'Banheiros': baths, 'Vagas': garage, 'Preço': price_sale, 'Condomínio': condominium_fee, 'IPTU': iptu, 'Caracteristicas': characteristics, 'Link': link}
        results.append(row)
        list_position += 1
        print(list_position)
        print(row)
        driver.quit()
    

[WDM] - Downloading: 100%|██████████| 6.80M/6.80M [00:00<00:00, 14.2MB/s]


2645
{'Título': 'Ponto Comercial para Alugar, 29 m² por R$ 3.500/Mês', 'Descrição': 'Endereço do imóvel: Loja n.05 do edifício Joana de Gusmão, a rua João Pinto, n.30, Centro, Florianópolis/SC.Loja possui 29m² de área, e conta com um banheiro.Loja fica em frente ao antigo terminal Cidade de Florianópolis, próximo a Prefeitura de Florianópolis (Procon e Pró Cidadão), ao lado da Praça XV.Valores:Condomínio: R$ 493,07 mensalIPTU: R$ 61,44 mensalTaxa de lixo: R$ 55,00 mensalSeguro incêndio: a consular.Para maiores informações, acesse o nosso site ou agende uma visita.48. - Megam Imobiliária', 'Endereço': 'Rua João Pinto', 'Número': '30', 'Bairro': 'Centro', 'Cidade': 'Florianópolis', 'Estado': 'SC', 'Área': '29m²', 'Quartos': 'Não informado Solicitar', 'Banheiros': 'Não informado Solicitar', 'Vagas': 'Não informado Solicitar', 'Preço': 'R$ 3.500/Mês', 'Condomínio': 'N/I', 'IPTU': 'N/I', 'Caracteristicas': 'N/I', 'Link': 'https://www.vivareal.com.br/imovel/ponto-comercial-centro-bairros-flo

WebDriverException: Message: unknown error: cannot determine loading status
from no such window
  (Session info: chrome=112.0.5615.138)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00A7DCE3+50899]
	(No symbol) [0x00A0E111]
	(No symbol) [0x00915450]
	(No symbol) [0x0090879A]
	(No symbol) [0x00907118]
	(No symbol) [0x0090778A]
	(No symbol) [0x00910B74]
	(No symbol) [0x0091AB28]
	(No symbol) [0x0091D4D6]
	(No symbol) [0x00907AE3]
	(No symbol) [0x0091A835]
	(No symbol) [0x0096DF77]
	(No symbol) [0x0095B6F6]
	(No symbol) [0x00937708]
	(No symbol) [0x0093886D]
	GetHandleVerifier [0x00CE3EAE+2566302]
	GetHandleVerifier [0x00D192B1+2784417]
	GetHandleVerifier [0x00D1327C+2759788]
	GetHandleVerifier [0x00B15740+672048]
	(No symbol) [0x00A18872]
	(No symbol) [0x00A141C8]
	(No symbol) [0x00A142AB]
	(No symbol) [0x00A071B7]
	BaseThreadInitThunk [0x76810099+25]
	RtlGetAppContainerNamedObjectPath [0x77477B6E+286]
	RtlGetAppContainerNamedObjectPath [0x77477B3E+238]


### 4) Verifica quantas partes nós temos daquele csv e salva como uma parte nova.

In [6]:
df = pd.DataFrame(results)

# Count the number of files starting with "dataframe_floripa" in the root folder
N = len([filename for filename in os.listdir() if filename.startswith("data_floripa_rent")])


# Save the dataframe to a CSV file with a name based on the count of files in the root folder
filename = f"data_floripa_rent_pt{N}.csv"
N 

1

In [7]:
df.to_csv(filename, index=False)

### 5) Unifica os CSVs presentes no diretório

In [15]:
import glob
# Get a list of all CSV files with the name "data_floripa_rent" in the root folder
csv_files = glob.glob('data_floripa_rent*.csv')

# Initialize an empty list to hold the individual dataframes
dfs = []

# Loop through each CSV file and append its contents to the list of dataframes
for file in csv_files:
    df = pd.read_csv(file)
    dfs.append(df)

# Concatenate all of the individual dataframes into a single unified dataframe
unified_df = pd.concat(dfs, ignore_index=True)
unified_df.to_csv("dataframe_floripa_rent_complete.csv", index=False) 

In [16]:
unified_df

,Título,Descrição,Endereço,Número,Bairro,Cidade,Estado,Área,Quartos,Banheiros,Vagas,Preço,Condomínio,IPTU,Caracteristicas,Link
0,"Sala Comercial e 1 banheiro para Alugar, 41 m²...","Sala comercial com área de 41 m², copa, lavabo...",Rua General Liberato Bittencourt,1914,Estreito,Florianópolis,SC,41m²,Não informado Solicitar,1 banheiro,Não informado Solicitar,R$ 670/Mês,R$ 574,R$ 217,Elevador Interfone Copa,https://www.vivareal.com.br/imovel/sala-comerc...
1,"Prédio Comercial e 6 banheiros à Venda, 450 m²...",Traga seu negócio para um dos lugares de grand...,Rua Anita Garibaldi,123,Centro,Florianópolis,SC,450m²,Não informado Solicitar,6 banheiros,6 vagas,R$ 4.000.000,R$ 5.526,N/I,N/I,https://www.vivareal.com.br/imovel/predio-come...
2,Apartamento com 1 Quarto e 2 banheiros para Al...,"Apartamento mobiliado no Centro com 1 suíte, l...",N/I,N/I,Centro,Florianópolis,SC,45m²,1 quarto,2 banheiros 1 suíte,1 vaga,R$ 3.400/Mês,R$ 551,R$ 90,Mobiliado Elevador Academia Ar-condicion...,https://www.vivareal.com.br/imovel/apartamento...
3,"Sala Comercial e 1 banheiro para Alugar, 42 m²...","Sala Comercial com 42m², dividido em 3 ambient...",N/I,N/I,Centro,Florianópolis,SC,42m²,Não informado Solicitar,1 banheiro,Não informado Solicitar,R$ 1.200/Mês,R$ 450,N/I,N/I,https://www.vivareal.com.br/imovel/sala-comerc...
4,"Casa com 1 Quarto e 1 banheiro para Alugar, 30...",Casa com aproximadamente 30 m² de área constru...,Rua João Alcântara da Cunha,59,Coqueiros,Florianópolis,SC,30m²,1 quarto,1 banheiro,1 vaga,R$ 1.600/Mês,N/I,R$ 60,Cozinha Área de serviço,https://www.vivareal.com.br/imovel/casa-1-quar...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,"Sala Comercial e 1 banheiro para Alugar, 72 m²...","Excelente e ampla sala com vaga de garagem , i...",Rua Bento Gonçalves,183,Centro,Florianópolis,SC,72m²,Não informado Solicitar,1 banheiro,1 vaga,R$ 2.500/Mês,R$ 690,R$ 118,Garagem,https://www.vivareal.com.br/imovel/sala-comerc...
158,"Sala Comercial e 4 banheiros para Alugar, 280 ...",Loja comercial com sobreloja para locação pron...,Rua Patrício Farias,N/I,Itacorubi,Florianópolis,SC,280m²,Não informado Solicitar,4 banheiros,4 vagas,R$ 14.000/Mês,R$ 1.874,R$ 956,N/I,https://www.vivareal.com.br/imovel/sala-comerc...
159,"Casa com 5 Quartos e 4 banheiros à Venda, 300 ...",A casa está localizado no bairro Jurerê Intern...,Rua dos Amborés,75,Jurerê Internacional,Florianópolis,SC,300m²,5 quartos,4 banheiros 2 suítes,3 vagas,R$ 4.300.000,N/I,N/I,Mobiliado Aceita animais Ar-condicionado ...,https://www.vivareal.com.br/imovel/casa-5-quar...
160,Apartamento com 2 Quartos e 2 banheiros para A...,"Apartamento 2 dormitórios sendo 1 suite , 1 va...",Rua Berta Lutz,30,Cachoeira do Bom Jesus,Florianópolis,SC,70m²,2 quartos,2 banheiros 1 suíte,1 vaga,R$ 2.800/Mês,R$ 800,R$ 180,Churrasqueira Sala de jantar Área de serviço,https://www.vivareal.com.br/imovel/apartamento...
